In [95]:
import os
import warnings
import pandas as pd
import numpy as np
import re 

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, regexp, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

warnings.filterwarnings('ignore')

In [99]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# need to only download only once
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chantal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/chantal/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Stats

Collect all the datasets

In [65]:
reviews = {}

# assuming naming follows 'type' + '_complete.csv' structure 
for f in os.listdir('data/'):
    if not f.startswith('.'):
        key = re.split(r'_', f)
        reviews[key[0]] = f
        
reviews

{'Scaling': 'Scaling_complete.csv',
 'Rehab': 'Rehab_complete.csv',
 'WASH': 'WASH_complete.csv',
 'ADIPP': 'ADIPP_complete.csv',
 'NCDS': 'NCDS_complete.csv',
 'VitaminD': 'VitaminD_complete.csv'}

In [ ]:
PATH = os.path.abspath('data')

for key, dataset in reviews.items():
    reviews[key] = pd.read_csv(os.path.join(PATH, dataset), encoding='latin1')

In [72]:
reviews['Scaling'].head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,DOI,Ref,Covidence #,Study,Notes,Tags,Inclusion
0,Is this scaling nonlinear? [arXiv],"Leitao, J.C.; Miotto, J.M.; Gerlach, M.; Altma...",One of the most celebrated findings in complex...,2016,NaN,arXiv,NaN,NaN,11-pp.,NaN,NaN,NaN,#9879,Leitao 2016,Takhliq Amir (2019-07-19 07:24:56)(Select): I ...,NaN,0
1,Scaling of foreign attractiveness for countrie...,"Bojic, Iva; Belyi, Alexander; Ratti, Carlo; So...","People's behavior on online social networks, w...",2016,NaN,Applied Geography,73,NaN,47-52,NaN,10.1016/j.apgeog.2016.06.006,NaN,#4381,Bojic 2016,Takhliq Amir (2019-07-18 13:50:05)(Select): Th...,NaN,0
2,Coastal vs inland sensitivity to desertificati...,"Salvati, Luca; Smiraglia, Daniela; Bajocco, So...",The present study assesses the spatial distrib...,2015,NaN,Rendiconti Lincei,26,NaN,571-576,NaN,10.1007/s12210-014-0339-4,NaN,#6536,Salvati 2015,NaN,NaN,0
3,Earthquake hazard and risk assessment based on...,"Kossobokov, V.G.; Nekrasova, A.K.",We apply the general concept of seismic risk a...,2018,NaN,Natural Hazards,93,3,1435-49,NaN,10.1007/s11069-018-3359-z,NaN,#9852,Kossobokov 2018,NaN,NaN,0
4,Dynamic evaluation of seismic hazard and risks...,"Kossobokov, Vladimir G.; Nekrasova, Anastasia",We continue applying the general concept of se...,2016,NaN,American Geophysical Union Fall Meeting,2016,NaN,NaN,NaN,NaN,NaN,#9495,Kossobokov 2016,NaN,NaN,0


Keep only relevant columns: Title, Abstract, Notes and Inclusion

In [74]:
to_keep = ['Title', 'Abstract', 'Notes', 'Inclusion']

for key, dataset in reviews.items():
    reviews[key] = dataset[to_keep]

In [80]:
reviews['Scaling'].head()

,Title,Abstract,Notes,Inclusion
0,Is this scaling nonlinear? [arXiv],One of the most celebrated findings in complex...,Takhliq Amir (2019-07-19 07:24:56)(Select): I ...,0
1,Scaling of foreign attractiveness for countrie...,"People's behavior on online social networks, w...",Takhliq Amir (2019-07-18 13:50:05)(Select): Th...,0
2,Coastal vs inland sensitivity to desertificati...,The present study assesses the spatial distrib...,NaN,0
3,Earthquake hazard and risk assessment based on...,We apply the general concept of seismic risk a...,NaN,0
4,Dynamic evaluation of seismic hazard and risks...,We continue applying the general concept of se...,NaN,0


Concatenate Title and Abstract fields

In [88]:
for key, dataset in reviews.items():
    reviews[key]['All_Text'] = reviews[key].apply(lambda x:
                                                         f"{x['Title']} {x['Abstract']}",
                                                         axis = 1)

Clean up and preprocess text: remove special characters, punctuation, tokenize, lemmatize, remove any repeated information (e.g., headings), replace NaNs with 0s

In [91]:
def clean_text(s):
    s = s.str.lower()                         # put to lowercase for homogeneity    
    s = s.str.replace(r'_', ' ')              # remove underscores from the notes
    s = s.str.replace(r'\W', ' ')             # remove punctutation
    stop = set(stopwords.words('english'))    # define stop words
    lemmatizer = WordNetLemmatizer()          # lemmatize - a lot of repeat words
    return s.apply(lambda x: [lemmatizer.lemmatize(word, 'v') 
                              for word in x.split() 
                              if word not in stop]) # remove stopwords

In [103]:
for key, dataset in reviews.items():
    reviews[key][['All_Text']].apply(lambda x: clean_text(x))

In [112]:
reviews['Scaling'][['All_Text']].apply(lambda x: clean_text(x))

,All_Text
0,"[scale, nonlinear, arxiv, one, celebrate, find..."
1,"[scale, foreign, attractiveness, countries, st..."
2,"[coastal, vs, inland, sensitivity, desertifica..."
3,"[earthquake, hazard, risk, assessment, base, u..."
4,"[dynamic, evaluation, seismic, hazard, risk, b..."
...,...
10687,"[empirical, analysis, connection, power, law, ..."
10688,"[positive, negative, feedbacks, free, scale, p..."
10689,"[growth, urban, extent, allometric, analysis, ..."
10690,"[influenza, pneumonia, mortality, 66, large, c..."


Count average length of abstracts, number of abstracts per dataset, count number of notes, maybe some simple correlations? 

# Keyword Extraction